<a href="https://colab.research.google.com/github/isamdr86/towards-ai/blob/main/notebooks/HF_Inference_ir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q requests huggingface_hub

In [3]:
# Set the following API Keys in the Python environment. Will be used later.
from google.colab import userdata
HF_API_TOKEN = userdata.get('huggingface_api_key')

In [4]:
from huggingface_hub import login

login(token=HF_API_TOKEN)

In [5]:
import requests


def query(payload, model_name):
    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {HF_API_TOKEN}"}

    response = requests.post(API_URL, headers=headers, json=payload)

    return response.json()

# API


## GPT-2 for Text Completion


In [6]:
data = query({"inputs": "Can you please let us know more details about"}, "gpt2")

In [7]:
data

[{'generated_text': 'Can you please let us know more details about our artists to accommodate the latest selection and provide free access for all their service. Supermedia flat sheet music is completed by the talented members of our crew No. 10 Rex Long, Main Producing P&B Beauclair from House of City of Heck. Our custom punched plexiglass sets are done by the exceptional engineers of House of City of Heck, featuring our custom cut logo vinyl designs from gutter details and left-of-center brick rock design from the iconic featured friend of Massive Hadley who leaves a billion miles a second. Then the custom pre-installed Trojan GPS units which automatically detect and control on KYC computer are installed on the tape, ready to connect the Media Center. HDMI,com TV services, Music and Music Streaming. All of its music, podcasts and podcasting meters pass through the huge CPU power of House of City of Heck to offer the intelligently designed 7.5 channel, channeled "Domain" broadcast ded

In [8]:
print(data[0]["generated_text"])

Can you please let us know more details about our artists to accommodate the latest selection and provide free access for all their service. Supermedia flat sheet music is completed by the talented members of our crew No. 10 Rex Long, Main Producing P&B Beauclair from House of City of Heck. Our custom punched plexiglass sets are done by the exceptional engineers of House of City of Heck, featuring our custom cut logo vinyl designs from gutter details and left-of-center brick rock design from the iconic featured friend of Massive Hadley who leaves a billion miles a second. Then the custom pre-installed Trojan GPS units which automatically detect and control on KYC computer are installed on the tape, ready to connect the Media Center. HDMI,com TV services, Music and Music Streaming. All of its music, podcasts and podcasting meters pass through the huge CPU power of House of City of Heck to offer the intelligently designed 7.5 channel, channeled "Domain" broadcast dedicated Sound Current 

## Llama-3 8B


In [ ]:
data = query(
    {"inputs": "Write a poem about blue sky."}, "meta-llama/Meta-Llama-3-8B-Instruct"
)

In [ ]:
data[0]["generated_text"]

## Summarization


In [ ]:
text_to_summarize = """Wall Street was quiet Friday ahead of the monthly jobs report.
S&P and Nasdaq futures were little changed and Dow futures were a couple of points lower.
It’s been a shortened week for traders, with markets closing early Wednesday and closed all day Thursday in observance of the July 4th holiday.
Investors are looking for another Goldilocks jobs report Friday that shows a gentle cooling-off of the labor market.
Wednesday’s ADP data, which showed that private payrolls fell to 150,000, created little reaction among traders
— but the government’s monthly tally will likely trigger more of a response."""

data = query({"inputs": text_to_summarize}, "facebook/bart-large-cnn")

In [ ]:
data[0]["summary_text"]

## Sentiment Analysis


In [ ]:
text_to_analyse = """I love how this app simplifies complex tasks \
effortlessly . I'm frustrated by the frequent errors in the software's \
latest update"""

data = query(
    {"inputs": text_to_summarize}, "distilbert-base-uncased-finetuned-sst-2-english"
)

In [ ]:
print(data[0])

## Example of Loading Model


In [ ]:
data

# Local


## Text-to-image


In [ ]:
!pip install -q -U diffusers transformers torch torchvision

In [ ]:
import transformers
transformers.utils.move_cache()

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

model_id = "stabilityai/stable-diffusion-2-1"

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

pipe = pipe.to("cuda")

prompt = """Photorealistic abandoned space station interior, shot on Sony A7SIII. Emergency lights casting red glow on metallic walls,
floating debris in zero gravity. Dust particles catching light, detailed control panels with authentic displays, worn metal
textures. Natural light from Earth visible through cracked viewport. Documentary photography style, high dynamic range."""

negative_prompt = "cartoon, anime, illustration, painted, artificial, oversaturated, low quality, blurry, noisy, text, watermark"

image = pipe(prompt = prompt,
             negative_prompt = negative_prompt,
             guidance_scale=7).images[0]

image.save("space_station.png")

In [ ]:
image

## Generate Text Embeddings


In [ ]:
!pip install -q sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

sentences = [
    """GAIA's questions are rooted in practical use cases, \
requiring AI systems to interact with a diverse and uncertain world, \
reflecting real-world applications.", " GAIA questions require accurate \
execution of complex sequences of actions, akin to the Proof of Work concept, \
where the solution is simple to verify but challenging to generate."""
]

model = SentenceTransformer("all-mpnet-base-v2", use_auth_token=HF_API_TOKEN)
embeddings = model.encode(sentences)
print(embeddings)

## Image Captioning


In [ ]:
from transformers import pipeline

image_to_text = pipeline(
    "image-to-text", model="nlpconnect/vit-gpt2-image-captioning", device="cuda"
)

caption = image_to_text("http://images.cocodataset.org/val2017/000000039769.jpg")

In [ ]:
print(caption[0]["generated_text"])

## Image Classification


In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()

In [ ]:
print("Predicted class:", model.config.id2label[predicted_class_idx])